### AB Model from spec

Package installation

In [ ]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [ ]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit

import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [ ]:
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Model parameters

In [ ]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 5
# no of css must be fixed
no_css = 5

Run Model

In [ ]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()

In [ ]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

In [ ]:
a = 26 % 24
print(a) 

Export results to CSV file

In [ ]:
model_run.datacollector.get_model_vars_dataframe().to_csv('06_03_5EV_agent_model_output.csv')

In [ ]:
# def unpack_datacollector(dc):
#     """
#     Unpacks the data from a DataCollector instance in Mesa.
    
#     Args:
#         dc (DataCollector): A DataCollector instance in Mesa.
    
#     Returns:
#         A dictionary with the keys as the variable names and the values as the lists of data for each variable.
#     """
#     data = {}
#     for varname, var in dc.model_vars.items():
#         data[varname] = var.get_data()
#     return data

### Data import and preprocessing

Helper functions. May eventually move to external module

In [ ]:
def split_column_values(df, col_name):
    """
    Splits the values in a column by commas and creates a new column for each value.

    Args:
        df (pandas.DataFrame): A pandas DataFrame.
        col_name (str): The name of the column to split.

    Returns:
        A pandas DataFrame with additional columns for each value in the input column.
    """
    # Get the unique values in the column
    unique_values = set(df[col_name].str.cat(sep=',').split(','))

    # Create a new column for each unique value
    for value in unique_values:
        df[value] = df[col_name].str.contains(value).astype(int)

    # Drop the original column
    df.drop(col_name, axis=1, inplace=True)

    return df


def unpack_and_join(df, column_name):
    """
    Unpacks the values in a column by commas and creates a new column for each value. Removes square brackets from the values.

    Args:
        df (pandas.DataFrame): A pandas DataFrame.
        column_name (str): The name of the column to unpack.

    Returns:
        A pandas DataFrame with additional columns for each value in the input column.
        
    """
    # Get the column values as a list of strings
    column_values = df[column_name].tolist()

    # Strip the square brackets from the strings
    column_values = [s.strip("[]") for s in column_values]

    # Split the strings on commas and create a list of lists
    split_values = [s.split(",") for s in column_values]

    # Get the number of columns needed
    num_cols = max([len(row) for row in split_values])

    # Create the new columns in the output dataframe
    column_names = [column_name+"_unpacked_"+str(i) for i in range(num_cols)]
    new_df = pd.DataFrame(columns=column_names)

    # Loop over the original column values and add the unpacked values to the new dataframe
    for vals in split_values:
        row_data = {}
        for i in range(num_cols):
            if i < len(vals):
                row_data[column_name+"_unpacked_"+str(i)] = vals[i].strip()
            else:
                row_data[column_name+"_unpacked_"+str(i)] = ""
        new_df = new_df.append(row_data, ignore_index=True)

    # Merge the original dataframe with the new unpacked dataframe
    merged_df = pd.concat([df, new_df], axis=1)

    return merged_df


Data import

In [ ]:
data = pd.read_csv('06_03_5EV_agent_model_output.csv')

In [ ]:
data.info()
# data.head()

In [ ]:
a = unpack_and_join(data, 'EVs Charge Level')
# print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

# def add_integer_column(df):
#     n_rows = df.shape[0] # get number of rows
#     df['new_col'] = pd.Series(range(n_rows)) # create new column with range of integers
#     return df

def add_time_col(df):
    n_rows = df.shape[0] # get number of rows
    df.insert(0, 'Timestep', pd.Series(range(n_rows))) # insert new column at index 0
    return df

def plot_data_scatter(df, x_col, y_col):
    """
    Plots data from two columns in a Pandas DataFrame using Plotly.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the data to be plotted.
    x_col : str
        The name of the column containing the x-axis data.
    y_col : str
        The name of the column containing the y-axis data.

    Returns:
    --------
    fig : plotly.graph_objs._figure.Figure
        The Plotly figure object containing the scatter plot.
    """
    fig = px.scatter(df, x=x_col, y=y_col)
    return fig

def plot_data_line(df, x_col, y_col):
    """
    Plots data from two columns in a Pandas DataFrame using Plotly.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the data to be plotted.
    x_col : str
        The name of the column containing the x-axis data.
    y_col : str
        The name of the column containing the y-axis data.

    Returns:
    --------
    fig : plotly.graph_objs._figure.Figure
        The Plotly figure object containing the line plot.
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[x_col], y=df[y_col], mode='lines'))
    fig.update_layout(title='Line Plot', xaxis_title=x_col, yaxis_title=y_col)
    return fig

def plot_data_heatmap(df, x_col, y_col, z_col):
    """
    Plots data from three columns in a Pandas DataFrame using Plotly.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the data to be plotted.
    x_col : str
        The name of the column containing the x-axis data.
    y_col : str
        The name of the column containing the y-axis data.
    z_col : str
        The name of the column containing the z-axis data.

    Returns:
    --------
    fig : plotly.graph_objs._figure.Figure
        The Plotly figure object containing the heatmap.
    """
    fig = go.Figure()
    fig.add_trace(go.Heatmap(x=df[x_col], y=df[y_col], z=df[z_col]))
    fig.update_layout(title='Heatmap', xaxis_title=x_col, yaxis_title=y_col)
    return fig

def plot_data_lines(df, x_col, y_cols):
    """
    Plots data from multiple columns in a Pandas DataFrame as multiple lines using Plotly.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the data to be plotted.
    x_col : str
        The name of the column containing the x-axis data.
    y_cols : list of str
        The names of the columns containing the y-axis data.

    Returns:
    --------
    fig : plotly.graph_objs._figure.Figure
        The Plotly figure object containing the multiple line plot.
    """
    fig = go.Figure()
    for y_col in y_cols:
        fig.add_trace(go.Scatter(x=df[x_col], y=df[y_col], mode='lines', name=y_col))

    fig.update_layout(title='Multiple Line Plot', xaxis_title=x_col, yaxis_title='Value')
    
    # Sort the y-axis in ascending order
    fig.update_yaxes(autorange="reversed")
    
    return fig


In [ ]:
add_time_col(newdf)
newdf.head()

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

### Scrapbook